# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteCurve.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')


## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [17]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def x_point(y,m,c):
    print('y,m,c values and isnan(y,c,m) and are:',y,m,c,math.isnan(y), math.isnan(c), math.isnan(m))
    if (m < 0.1 and m > -0.1) or math.isnan(y) or math.isnan(c) or math.isnan(m):
        x = 1000
    else:
        x = (y-c)/m
        
    if(math.isnan(x)):
        x=1000
    return int(x);

def draw_lines(img, lines, thickness, color=[255, 0, 0]):
    image1=img
    image2=img
    m_left_sum = -5
    c_left_sum = 5
    left_weight = 5
    m_right_sum = 5
    c_right_sum = 5
    right_weight = 5
    min_y_left = img.shape[1];
    min_y_right = img.shape[1];
    color2=[0,0,225]
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            weight = (y2-y1)^2 + (x2-x1)^2
            intercept = y1 - slope*x1
            if slope < -0.5:
                m_left_sum = m_left_sum + weight*slope
                c_left_sum = c_left_sum + weight*intercept
                left_weight = weight + left_weight
                min_y_left = min(y1,y2,min_y_left);
            if slope > 0.5:
                m_right_sum = m_right_sum + weight*slope
                c_right_sum = c_right_sum + weight*intercept
                right_weight = weight + right_weight 
                min_y_right = min(y1,y2,min_y_right);
            cv2.line(image1, (x1, y1), (x2, y2), color2, thickness)
    m_left = m_left_sum/left_weight
    c_left = c_left_sum/left_weight
    m_right = m_right_sum/right_weight
    c_right = c_right_sum/right_weight
    #left lane
    if m_left < -0.5 and math.isnan(m_left)==0:
        cv2.line(image1, (x_point(img.shape[1],m_left,c_left), img.shape[1]), (x_point(min_y_left,m_left,c_left), min_y_left), color, thickness*5)
    #right lane
    if m_right > 0.5 and math.isnan(m_left)==0:
        cv2.line(image1, (x_point(img.shape[1],m_right,c_right), img.shape[1]), (x_point(min_y_right,m_right,c_right), min_y_right), color, thickness*5)
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(image1, (x1, y1), (x2, y2), color2, thickness)
    return image1   

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, thickness):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness)
    return line_img



# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [8]:
def draw_lanes(input_image):
    #grayscale the image
    gray=grayscale(input_image) 
    
    #gaussian blur with 
    kernel_size =3
    blur_gray=gaussian_blur(gray,kernel_size)
    
    #canny the image with 
    low_threshold = 50 
    high_threshold = 150
    canny_gray=canny(blur_gray,low_threshold,high_threshold)

    #mask the vertices
    # This time we are defining a four sided polygon to mask
    imshape = input_image.shape
    vertices = np.array([[(0.05*imshape[1],imshape[0]),
                          (0.40*imshape[1], imshape[0]*0.6), 
                          (0.6*imshape[1], imshape[0]*0.6), 
                          (0.95*imshape[1],imshape[0])]], dtype=np.int32)
    masked_canny=region_of_interest(canny_gray,vertices)

    #get hough_lines with rho=2, theta=np.pi/180, threshold = 20, 
    #min_line_len = 40, max_line_gap=10
    theta = np.pi/180
    line_image = hough_lines(masked_canny, 2, theta, 20, 40, 10,5)


    # Paint the lines with a 0.8 weight
    painted_image = weighted_img(line_image,input_image)
    return painted_image


    

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
from PIL import Image
import os
os.listdir("test_images/")
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.

inpath = "test_images/"
outpath = "output_test_images/"
valid_images = [".jpg",".gif",".png"]#,".tga"]
for filename in os.listdir("test_images/"):
    ext = os.path.splitext(filename)[1]
    if ext.lower() not in valid_images:
        continue
    #imgs.append(Image.open(os.path.join(path,f))
    image = mpimg.imread(os.path.join(inpath,filename))
    im = Image.fromarray(draw_lanes(image))
    im.save(os.path.join(outpath,'out_'+filename))





## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [4]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result=draw_lanes(image)

    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [18]:
challenge_output = 'test_videos_output/challenge.mp4'
clip2 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

Left and Right weight, m_left,c_left,m_right,c_right is -671 426 -0.660046352682 860.130313658 0.607140480273 24.8477056386
y,m,c values and isnan(y,c,m) and are: 1280 -0.660046352682 860.130313658 False False False
y,m,c values and isnan(y,c,m) and are: 434 -0.660046352682 860.130313658 False False False
y,m,c values and isnan(y,c,m) and are: 1280 0.607140480273 24.8477056386 False False False
y,m,c values and isnan(y,c,m) and are: 578 0.607140480273 24.8477056386 False False False
[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4






  0%|          | 0/251 [00:00<?, ?it/s]



  1%|          | 3/251 [00:00<00:09, 26.71it/s]

Left and Right weight, m_left,c_left,m_right,c_right is -671 426 -0.660046352682 860.130313658 0.607140480273 24.8477056386
y,m,c values and isnan(y,c,m) and are: 1280 -0.660046352682 860.130313658 False False False
y,m,c values and isnan(y,c,m) and are: 434 -0.660046352682 860.130313658 False False False
y,m,c values and isnan(y,c,m) and are: 1280 0.607140480273 24.8477056386 False False False
y,m,c values and isnan(y,c,m) and are: 578 0.607140480273 24.8477056386 False False False
Left and Right weight, m_left,c_left,m_right,c_right is -847 432 -0.657537376654 858.295534435 0.615222797287 13.0396194945
y,m,c values and isnan(y,c,m) and are: 1280 -0.657537376654 858.295534435 False False False
y,m,c values and isnan(y,c,m) and are: 433 -0.657537376654 858.295534435 False False False
y,m,c values and isnan(y,c,m) and are: 1280 0.615222797287 13.0396194945 False False False
y,m,c values and isnan(y,c,m) and are: 471 0.615222797287 13.0396194945 False False False
Left and Right weight, m





  2%|▏         | 5/251 [00:00<00:10, 23.95it/s]

Left and Right weight, m_left,c_left,m_right,c_right is -1011 5 -0.670292855935 866.280793207 1.0 1.0
y,m,c values and isnan(y,c,m) and are: 1280 -0.670292855935 866.280793207 False False False
y,m,c values and isnan(y,c,m) and are: 437 -0.670292855935 866.280793207 False False False
y,m,c values and isnan(y,c,m) and are: 1280 1.0 1.0 False False False
y,m,c values and isnan(y,c,m) and are: 1280 1.0 1.0 False False False


...





 98%|█████████▊| 247/251 [00:24<00:00,  8.70it/s]



 99%|█████████▉| 249/251 [00:24<00:00,  8.87it/s]

y,m,c values and isnan(y,c,m) and are: 563 0.576194947447 50.0414769676 False False False
Left and Right weight, m_left,c_left,m_right,c_right is -479 710 -0.740841134096 914.208853068 0.575098413156 50.0315376436
y,m,c values and isnan(y,c,m) and are: 1280 -0.740841134096 914.208853068 False False False
y,m,c values and isnan(y,c,m) and are: 439 -0.740841134096 914.208853068 False False False
y,m,c values and isnan(y,c,m) and are: 1280 0.575098413156 50.0315376436 False False False
y,m,c values and isnan(y,c,m) and are: 585 0.575098413156 50.0315376436 False False False
Left and Right weight, m_left,c_left,m_right,c_right is -747 305 -0.729000894005 904.130964187 0.591964802314 36.5786282546
y,m,c values and isnan(y,c,m) and are: 1280 -0.729000894005 904.130964187 False False False
y,m,c values and isnan(y,c,m) and are: 437 -0.729000894005 904.130964187 False False False
y,m,c values and isnan(y,c,m) and are: 1280 0.591964802314 36.5786282546 False False False
y,m,c values and isnan(y





100%|█████████▉| 250/251 [00:24<00:00,  8.71it/s]



100%|██████████| 251/251 [00:24<00:00, 10.21it/s]

Left and Right weight, m_left,c_left,m_right,c_right is -1044 5 -0.713769826825 891.610689083 1.0 1.0
y,m,c values and isnan(y,c,m) and are: 1280 -0.713769826825 891.610689083 False False False
y,m,c values and isnan(y,c,m) and are: 437 -0.713769826825 891.610689083 False False False
y,m,c values and isnan(y,c,m) and are: 1280 1.0 1.0 False False False
y,m,c values and isnan(y,c,m) and are: 1280 1.0 1.0 False False False
Left and Right weight, m_left,c_left,m_right,c_right is -959 5 -0.732780615305 904.272296465 1.0 1.0
y,m,c values and isnan(y,c,m) and are: 1280 -0.732780615305 904.272296465 False False False
y,m,c values and isnan(y,c,m) and are: 445 -0.732780615305 904.272296465 False False False
y,m,c values and isnan(y,c,m) and are: 1280 1.0 1.0 False False False
y,m,c values and isnan(y,c,m) and are: 1280 1.0 1.0 False False False


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 11.2 s, sys: 4.05 s, total: 15.2 s
Wall time: 27.5 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))